In [58]:
import pandas as pd
import numpy as np
import json
from scipy import stats
import tqdm

# Funzioni

In [59]:
def nclusters(data, threshold):
    data = [float(el) for el in data]
    data = sorted(data)
    start = data[0]
    max_val = start + threshold
    c = (start, max_val)
    cluster = dict()
    for i in data:
        if i <= max_val:
            if c in cluster.keys():
                cluster[c] += 1
            else:
                cluster[c] = 1
        else:
            max_val = i + threshold
            c = (i, max_val)
            cluster[c] = 1
    #ora ho il dizionario con i cluster di una run
    C_num = len(data)**2
    C_den = 0
    for k in cluster.keys():
        C_den += cluster[k]*cluster[k]
    C = C_num / C_den
    return C

def entropy(opinions, n, nbins):
    bincounts, bin_edges = np.histogram(opinions, bins = np.linspace(0, 1, nbins))
    probabilities = bincounts/n
    entr = stats.entropy(probabilities)
    return entr

def nodeperc_withininterval(opinions, center, width):
    sortedops = sorted(opinions)
    sortedopsarr = np.array(sortedops)
    withinrange = np.where(np.logical_and(sortedopsarr>=(center-width), sortedopsarr<=(center+width)))
    withinrange = withinrange[0]
    perc = len(withinrange) / len(sortedops)
    return perc

def average_opinions(opinions):
    a = np.array(opinions)
    return np.average(a)

def std_opinions(opinions):
    a = np.array(opinions)
    return np.std(a)

def median_opinions(opinions):
    a = np.array(opinions)
    return np.median(a)
maxentr10 = entropy(np.random.uniform(0.0, 1.0, 250), 250, 11)

In [76]:
model = "rewiring"

In [77]:
finalfile = f"{model} results 100.csv"

In [78]:
#create one aggregate file
import os
aggrfile = open(f"aggregate/{finalfile}", "w+")
header = "model,graph,n,p,pr,eps,gam,max_it\n"
aggrfile.write(header)
aggrfile.close()
aggrfile = open(f"aggregate/{finalfile}", "a+")
for graph in ['er']:
    if graph == "er":
        p = 0.1
    else:
        p=5
    for pr in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for eps in [0.35]:
            for gam in [0.0, 0.5, 1.0, 1.5]:
                name = f"{model} {graph}{p} pr{pr} e{eps} g{gam} mi1000000"
                s = f"{model},{graph},100,{p},{pr},{eps},{gam},1000000\n"
                aggrfile.write(s)
aggrfile.close()

In [79]:
results = pd.read_csv(f"aggregate/{finalfile}")
results

,model,graph,n,p,pr,eps,gam,max_it
0,rewiring,er,100,0.1,0.0,0.35,0.0,1000000
1,rewiring,er,100,0.1,0.0,0.35,0.5,1000000
2,rewiring,er,100,0.1,0.0,0.35,1.0,1000000
3,rewiring,er,100,0.1,0.0,0.35,1.5,1000000
4,rewiring,er,100,0.1,0.1,0.35,0.0,1000000
5,rewiring,er,100,0.1,0.1,0.35,0.5,1000000
6,rewiring,er,100,0.1,0.1,0.35,1.0,1000000
7,rewiring,er,100,0.1,0.1,0.35,1.5,1000000
8,rewiring,er,100,0.1,0.2,0.35,0.0,1000000
9,rewiring,er,100,0.1,0.2,0.35,0.5,1000000


In [80]:
avg = []
std = []
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi1000000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_iterations {name}.json')
    data = json.load(jsonfile)
    finalitarr = []
    for nr in data.keys():
        finalitarr.append(data[nr])
    finalitarr = np.array(finalitarr)
    avg.append(np.average(finalitarr))
    std.append(np.std(finalitarr))
results["avg_nit"] = avg
results["std_nit"] = std
results["nruns"] = int(nr)+1

doing rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.5 e0.35 g0.0 mi

In [81]:
def iqm(arr):
    tf = np.percentile(arr, 25)
    sf = np.percentile(arr, 75)
    tmp = []
    for el in arr:
        if el < min(tf, sf) or el > max(tf, sf):
            continue
        else:
            tmp.append(el)
    return np.array(tmp)
    

avg = []
std = []
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi1000000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_iterations {name}.json')
    data = json.load(jsonfile)
    finalitarr = []
    for nr in data.keys():
        finalitarr.append(data[nr])
    finalitarr = np.array(finalitarr)
    avg.append(np.average(iqm(finalitarr)))
    std.append(np.std(iqm(finalitarr)))
    
results["iqm_nit"] = avg
results["iqm_std_nit"] = std
results["nruns"] = int(nr)+1

doing rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.5 e0.35 g0.0 mi

In [82]:
avg01 = []
std01 = []
avg001=[]
std001=[]
avg00001=[]
std00001=[]
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi1000000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    ncarray01 = []
    ncarray001 = []
    ncarray00001 = []
    for nr in data.keys():
        finalops = list(data[nr])
        nc01 = nclusters(finalops, 0.1)
        nc001 = nclusters(finalops, 0.01)
        nc00001 = nclusters(finalops, 0.0001)
        ncarray01.append(nc01)
        ncarray001.append(nc001)
        ncarray00001.append(nc00001)
    ncarray01 = np.array(ncarray01)
    ncarray001 = np.array(ncarray001)
    ncarray00001 = np.array(ncarray00001)
    avg01.append(np.average(ncarray01))
    std01.append(np.std(ncarray01))
    avg001.append(np.average(ncarray001))
    std001.append(np.std(ncarray001))
    avg00001.append(np.average(ncarray00001))
    std00001.append(np.std(ncarray00001))
results["avg_ncluster_001"] = avg001
results["std_ncluster_001"] = std001

doing rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.5 e0.35 g0.0 mi

In [83]:
results.to_csv(f"aggregate/{finalfile}")

In [84]:
results = pd.read_csv(f"aggregate/{finalfile}", index_col=[0])
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,nruns,iqm_nit,iqm_std_nit,avg_ncluster_001,std_ncluster_001
0,rewiring,er,100,0.1,0.0,0.35,0.0,1000000,42.633333,6.848276,30,41.400000,2.122891,1.035865,0.042939
1,rewiring,er,100,0.1,0.0,0.35,0.5,1000000,148.500000,61.193545,30,131.642857,7.649530,1.086109,0.200662
2,rewiring,er,100,0.1,0.0,0.35,1.0,1000000,8615.533333,2535.349059,30,8622.785714,1354.539467,1.410969,0.433416
3,rewiring,er,100,0.1,0.0,0.35,1.5,1000000,2766.633333,2816.908299,30,2001.866667,1209.562228,24.240904,2.797412
4,rewiring,er,100,0.1,0.1,0.35,0.0,1000000,296.833333,200.935493,30,258.214286,73.455700,1.122025,0.203903


In [85]:
averageops = []
averagestdops = []
averagemedianops = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi1000000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    avgarray = []
    stdarray = []
    medarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        avg = average_opinions(finalops)
        med = median_opinions(finalops)
        std = std_opinions(finalops)
        avgarray.append(avg)
        medarray.append(med)
        stdarray.append(std)
    avgarray = np.array(avgarray)
    medarray = np.array(medarray)
    stdarray = np.array(stdarray)
    averageops.append(np.average(avgarray))
    averagestdops.append(np.average(stdarray))
    averagemedianops.append(np.average(medarray))
results["avg_mean_opinion"] = averageops
results["avg_std_opinions"] = averagestdops
results["avg_median_opinion"] = averagemedianops

doing rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.5 e0.35 g0.0 mi

In [86]:
results.to_csv(f"aggregate/{finalfile}")

In [87]:
results = pd.read_csv(f"aggregate/{finalfile}", index_col=[0])
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,nruns,iqm_nit,iqm_std_nit,avg_ncluster_001,std_ncluster_001,avg_mean_opinion,avg_std_opinions,avg_median_opinion
0,rewiring,er,100,0.1,0.0,0.35,0.0,1000000,42.633333,6.848276,30,41.400000,2.122891,1.035865,0.042939,0.504831,0.047380,0.507131
1,rewiring,er,100,0.1,0.0,0.35,0.5,1000000,148.500000,61.193545,30,131.642857,7.649530,1.086109,0.200662,0.498653,0.049228,0.509392
2,rewiring,er,100,0.1,0.0,0.35,1.0,1000000,8615.533333,2535.349059,30,8622.785714,1354.539467,1.410969,0.433416,0.497057,0.125289,0.464669
3,rewiring,er,100,0.1,0.0,0.35,1.5,1000000,2766.633333,2816.908299,30,2001.866667,1209.562228,24.240904,2.797412,0.488412,0.274289,0.473872
4,rewiring,er,100,0.1,0.1,0.35,0.0,1000000,296.833333,200.935493,30,258.214286,73.455700,1.122025,0.203903,0.500419,0.072052,0.498995


In [88]:
values_avg = []
values_std = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi1000000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    entrarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        entr = entropy(finalops, 100, 11)
        entrarray.append(entr)
    entrarray = np.array(entrarray)
    values_avg.append(np.average(entrarray))
    values_std.append(np.std(entrarray))

results["10B_avg_entr"] = values_avg
results["10B_std_entr"] = values_std

results['10B_avg_norm_entr'] = results['10B_avg_entr'].apply(lambda x: x/maxentr10)

doing rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.0 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.1 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.2 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.3 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g0.0 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g0.5 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g1.0 mi1000000
doing rewiring er0.1 pr0.4 e0.35 g1.5 mi1000000
doing rewiring er0.1 pr0.5 e0.35 g0.0 mi

In [89]:
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,...,iqm_nit,iqm_std_nit,avg_ncluster_001,std_ncluster_001,avg_mean_opinion,avg_std_opinions,avg_median_opinion,10B_avg_entr,10B_std_entr,10B_avg_norm_entr
0,rewiring,er,100,0.1,0.0,0.35,0.0,1000000,42.633333,6.848276,...,41.400000,2.122891,1.035865,0.042939,0.504831,0.047380,0.507131,0.084548,0.087226,0.036834
1,rewiring,er,100,0.1,0.0,0.35,0.5,1000000,148.500000,61.193545,...,131.642857,7.649530,1.086109,0.200662,0.498653,0.049228,0.509392,0.110746,0.174919,0.048248
2,rewiring,er,100,0.1,0.0,0.35,1.0,1000000,8615.533333,2535.349059,...,8622.785714,1354.539467,1.410969,0.433416,0.497057,0.125289,0.464669,0.374640,0.293124,0.163216
3,rewiring,er,100,0.1,0.0,0.35,1.5,1000000,2766.633333,2816.908299,...,2001.866667,1209.562228,24.240904,2.797412,0.488412,0.274289,0.473872,2.188860,0.054336,0.953600
4,rewiring,er,100,0.1,0.1,0.35,0.0,1000000,296.833333,200.935493,...,258.214286,73.455700,1.122025,0.203903,0.500419,0.072052,0.498995,0.186334,0.152060,0.081178


In [90]:
results.to_csv("aggregate/{}".format(finalfile))
results = pd.read_csv("aggregate/{}".format(finalfile), index_col=[0])

In [91]:
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,...,iqm_nit,iqm_std_nit,avg_ncluster_001,std_ncluster_001,avg_mean_opinion,avg_std_opinions,avg_median_opinion,10B_avg_entr,10B_std_entr,10B_avg_norm_entr
0,rewiring,er,100,0.1,0.0,0.35,0.0,1000000,42.633333,6.848276,...,41.400000,2.122891,1.035865,0.042939,0.504831,0.047380,0.507131,0.084548,0.087226,0.036834
1,rewiring,er,100,0.1,0.0,0.35,0.5,1000000,148.500000,61.193545,...,131.642857,7.649530,1.086109,0.200662,0.498653,0.049228,0.509392,0.110746,0.174919,0.048248
2,rewiring,er,100,0.1,0.0,0.35,1.0,1000000,8615.533333,2535.349059,...,8622.785714,1354.539467,1.410969,0.433416,0.497057,0.125289,0.464669,0.374640,0.293124,0.163216
3,rewiring,er,100,0.1,0.0,0.35,1.5,1000000,2766.633333,2816.908299,...,2001.866667,1209.562228,24.240904,2.797412,0.488412,0.274289,0.473872,2.188860,0.054336,0.953600
4,rewiring,er,100,0.1,0.1,0.35,0.0,1000000,296.833333,200.935493,...,258.214286,73.455700,1.122025,0.203903,0.500419,0.072052,0.498995,0.186334,0.152060,0.081178


Unused functions

# def nclusters_meanshift(data, bw):
#     ops = {i: data[i] for i in range(len(data))}
#     sorted_ops = sorted(ops.items(), key = lambda kv:(kv[1], kv[0]))
#     A=np.array([el[1] for el in sorted_ops]).reshape(-1,1)
#     clustering = MeanShift(bandwidth=bw).fit(A)
#     lbls = clustering.labels_
#     labels = np.arange(len(sorted_ops))
#     for i in range(len(labels)):
#         cl = lbls[i]
#         labels[sorted_ops[i][0]]=cl
#     cluster_participation_dict = {}
#     for l in labels:
#         if l not in cluster_participation_dict:
#             cluster_participation_dict[l] = 1
#         else:
#             cluster_participation_dict[l] += 1
#     #computing effective number of clusters using function explained in the paper
#     C_num = 0
#     C_den = 0
#     for k in cluster_participation_dict:
#         C_num += cluster_participation_dict[k]
#         C_den += ((cluster_participation_dict[k])**2)
#     C_num = (C_num**2)
#     C = C_num/C_den
#     return C

In [ ]:
def steady_state_coevolving(model, name, max_iterations=100000, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=False):
    if not os.path.exists(f"snapshotGraphs/edgelist {name} 0.csv"):
        print(type(model.graph.graph))
        system_status = []
        steady_it = 0
        for it in tqdm.tqdm(range(0, max_iterations), disable=not progress_bar):            
            its = model.iteration(node_status)
            
            if it == 0:
                G = model.graph.graph
                nx.write_edgelist(G, f"snapshotGraphs/edgelist {name} {it}.csv", delimiter=",")
                with open(f"snapshotGraphs/opinions {name} {it}.txt", "w") as opfile:
                    first_values = list(its['status'].values())
                    for op in first_values:
                        opfile.write(str(op)+"\n")

            elif it > 0:
                old = np.array(list(system_status[-1]['status'].values()))
                actual = np.array(list(its['status'].values()))
                # res = np.abs(old - actual)
                if np.all((its['max_diff'] < sensibility)):
                    steady_it += 1
                else:
                    steady_it = 0

                if it % 500 == 0:
                    G = model.graph.graph
                    nx.write_edgelist(G, f"snapshotGraphs/edgelist {name} {it}.csv", delimiter=",")
                    with open(f"snapshotGraphs/opinions {name} {it}.txt", "w") as opfile:
                        for op in list(actual):
                            opfile.write(str(op)+"\n")

            system_status.append(its)
            
            if steady_it == nsteady:
                G = model.graph.graph
                nx.write_edgelist(G, f"snapshotGraphs/edgelist {name} {it}.csv", delimiter=",")
                with open(f"snapshotGraphs/opinions {name} {it}.txt", "w") as opfile:
                    for op in list(actual):
                        opfile.write(str(op)+"\n")
                        
                return system_status[:-nsteady]

        return system_status
# Save snapshots of graph during evolution
import sys
sys.path.append("/home/pansanella/mydata/GitHub/local_packages/")
sys.path.append("/data1/users/pansanella/mydata/GitHub/local_packages/")
sys.path.append("/data1/users/pansanella/mydata/GitHub/local_packages/netdspatch_local/")

import json
import networkx as nx
import ndlib_local.ndlib.models.ModelConfig as mc
import ndlib_local.ndlib.models.opinions as op
import warnings
warnings.filterwarnings("ignore")
n = 250
max_it = 100000
for graphname in ['er', 'ba']:
    if graphname == 'er':
        p = 0.1
        graph = nx.erdos_renyi_graph(n, p)
    else:
        p = 5
        graph = nx.barabasi_albert_graph(n, p)
    for pr in [0.5]:
        for e in [0.2, 0.3, 0.4]:
            for g in [0.0, 0.5, 1.0, 1.5]:
                name = f"rewiring {graphname}{p} pr{pr} e{e} g{g} mi{max_it}"
                model = op.AdaptiveAlgorithmicBiasModel(graph)
                config = mc.Configuration()
                config.add_model_parameter("epsilon", e)
                config.add_model_parameter("gamma", g)
                config.add_model_parameter("p", pr)
                model.set_initial_status(config)
                steady_status = steady_state_coevolving(model=model, name=name, max_iterations=max_it, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=True)                                